In [ ]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import string
from clean_data import*

In [ ]:
from add_summary import*


In [ ]:
from find_category import*

In [4]:
candidate=DataFrame(columns=["id", "name","state_fk","date_birth", "place_birth", "position", "URL_photo", "party"])

candidate1=pd.DataFrame([[1, "Don Blankenship",1, "March 14, 1950", "Stopover, Kentucky", "Coal magnate", "https://i2.wp.com/woay.tv/wp-content/uploads/2017/05/GTY_don_blankenship_jt_160406_4x3_992.jpg?resize=735%2C400&ssl=1","Republican" ],
                     [2, "Bo Copley",1, "", " Dingess, West Virginia", "","https://heavyeditorial.files.wordpress.com/2017/05/bo-copley-e1494012597984.jpg?quality=65&strip=all&w=780&strip=all","Republican" ],
                     [3, "Evan Jenkins",1, "September 12, 1960", "Huntington, West Virginia","U.S. House of Representatives","https://www.congress.gov/img/member/114_rp_wv_3_jenkins_evan.jpg","Republican"  ],
                     [4, "Joe Manchin",1, " August 24, 1947", "Farmington, West Virginia", "United States Senator", "https://fi.wikipedia.org/wiki/Joe_Manchin#/media/File:Senator_Manchin.jpg", "Democrat"],
                     [5, "Patrick Morrisey",1,"December 21, 1967","Brooklyn, New York City, NY","West Virginia Attorney General since 2013","http://wvmetronews.com/wordpress/wp-content/uploads/2016/02/Patrick-Morrisey.jpg?x43308","Republican"  ],
                     [6, "Jack Newbrough",1, "", "","U.S. Navy veteran","http://s3.amazonaws.com/ogden_images/www.weirtondailytimes.com/images/2017/09/23190845/Newbrough-003-1100x825.jpg" ,"Republican" ],
                     [7, "Paula Jean Swearengin", 1, "born 1973/74","Mullens, West Virginia","","https://cbbba9d899c5f8ae6108-e379174d4d25dcb78174c6cab0a88cea.ssl.cf1.rackcdn.com/1275564-l.jpg", "Democrat"],
                     [8, "Tom Willis",1, "","","","https://api.ballotpedia.org/v3/thumbnail/200/300/crop/best/2C512069-7298-4BAC-B9F3-6BFECDC2FB57.jpeg","Republican" ],
                     [9, "Nicholas Freitas", 2, "", "", "","","Republican"],
                     [10, "E. W. Jackson", 2,"","","","","Republican"],
                     [11, "Tim Kaine", 2, "", "", "", "", "Democrat"],
                     [12, "Corey Stewart", 2, "", "", "", "", "Republican"],
                     [13, "Matt Waters", 2, "", "", "", "", "Libertarian"],
                     [14, "Carl Bible", 3, "", "", "", "", "Independent"],
                     [15, "Ted Cruz", 3, "", "", "", "", "Republican"],
                     [16, "Bob McNeil", 3, "", "", "", "", "American Citizen Party"],
                     [17, "Beto O'Rourke", 3, "", "", "", "", "Democrat"]],
                        columns=["id", "name","state_fk","date_birth", "place_birth", "position","URL_photo", "party"])
candidate=candidate.append(candidate1)


In [5]:
all_candidates = []
for index,row in candidate.iterrows():
    all_candidates.append(row['name'])

In [6]:
## script is hardcoded to take in data from the RAW_DATA file
# this function takes in a list of candidates (string form) and a name of the output json file
structure_data(all_candidates, "ALL_DATA")

In [7]:
# this script takes an input file and returns a dataframe of the file + summaries
all_data = add_summary("ALL_DATA.json")

In [ ]:
all_data

In [ ]:
b = [[2*x for y in a] for x in y]
c = [[2*x for x in y] for y in a]
#[[int(x) for x in feet] for x in feet]

In [ ]:
## category analysis

In [8]:
corpus = []
for index,row in all_data.iterrows():
    corpus.append(row['article_text'])

In [9]:
def find_categories_and_top20(corpus):
    corpus_sentences = [splitToSentences(x) for x in corpus]
    corpus_words = []
    for x in corpus_sentences:
        corpus_words.append(([convertToLowercase(lemmatizeWords(splitToWords(removePunctuation(y)))) for y in x]))
    #corpus_words = [removeStopWords(WhitespaceTokenizer().tokenize(x)) for x in corpus]

    final_corpus = []
    for x in corpus_words:
        final_corpus.append([item for sublist in x for item in removeStopWords(sublist) if item.isalpha()])
    return final_corpus


In [10]:
import time

def create_lda_page(final_corpus):
    start_time = time.time()

    dictionary = corpora.Dictionary(final_corpus)

    # Converts dictionary into a bag-of-words.
    corpusVec = [dictionary.doc2bow(text) for text in final_corpus]

    # Generate LDA model
    ldamodel = gensim.models.ldamodel.LdaModel(corpusVec, num_topics=10, id2word = dictionary, passes=500)
    # print(ldamodel.print_topics(num_topics=5, num_words=5))

    vis_data = pyLDAvis.gensim.prepare(ldamodel, corpusVec, dictionary)

    pyLDAvis.save_html(vis_data, 'lda_reduced_2.html')
    print("--- %s seconds ---" % (time.time() - start_time))

In [11]:
# call script to generate categories and 
relevant_words = find_categories_and_top20(corpus)

In [12]:
create_lda_page(relevant_words)

/Users/sarahmathew/Envs/capstone/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


--- 716.7678558826447 seconds ---


In [15]:
extra_stopwords = ["like", "it’s", "uh", "going", "that’s", "think", "actually", "kind", "…", "know", "come", "u", "really"
                   ,"mr", "june", "july", "august", "aug", "festival", "theater", "music", "concert", "dance", "performance"
                   ,"jazz", "band", "arts", "art","museum","painting", "artist", "exhibition","gallery", "ms", "sculpture"
                   ,"tell", "also","thats", "im", "album", "film", "movie", "opera", "cookbook", "orchestra", "play"
                   ,"street", "design", "photograph", "drawing", "collection", "street", "design", "direct", "jan", "may","summer"
                   ,"season", "production","park"]

In [16]:
extra_stopwords

['like',
 'it’s',
 'uh',
 'going',
 'that’s',
 'think',
 'actually',
 'kind',
 '…',
 'know',
 'come',
 'u',
 'really',
 'mr',
 'june',
 'july',
 'august',
 'aug',
 'festival',
 'theater',
 'music',
 'concert',
 'dance',
 'performance',
 'jazz',
 'band',
 'arts',
 'art',
 'museum',
 'painting',
 'artist',
 'exhibition',
 'gallery',
 'ms',
 'sculpture',
 'tell',
 'also',
 'thats',
 'im',
 'album',
 'film',
 'movie',
 'opera',
 'cookbook',
 'orchestra',
 'play',
 'street',
 'design',
 'photograph',
 'drawing',
 'collection',
 'street',
 'design',
 'direct',
 'jan',
 'may',
 'summer',
 'season',
 'production',
 'park']